# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921679369895                   -0.69    5.6         
  2   -7.926168301396       -2.35       -1.22    1.0    247ms
  3   -7.926837028023       -3.17       -2.37    1.6    268ms
  4   -7.926861519652       -4.61       -3.03    2.9    297ms
  5   -7.926861642000       -6.91       -3.39    2.0    290ms
  6   -7.926861669457       -7.56       -3.78    1.9    255ms
  7   -7.926861680427       -7.96       -4.33    1.2    239ms
  8   -7.926861681760       -8.88       -4.85    1.9    255ms
  9   -7.926861681861       -9.99       -5.29    2.0    294ms
 10   -7.926861681871      -11.00       -5.94    1.8    251ms
 11   -7.926861681873      -11.96       -7.12    2.1    276ms
 12   -7.926861681873      -13.65       -7.40    3.6    326ms
 13   -7.926861681873      -15.05       -8.28    1.4    258ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921662864079                   -0.69    6.0         
  2   -7.926163813212       -2.35       -1.22    1.0    247ms
  3   -7.926837681772       -3.17       -2.37    1.9    276ms
  4   -7.926861530130       -4.62       -3.04    2.6    345ms
  5   -7.926861646789       -6.93       -3.42    1.6    245ms
  6   -7.926861670526       -7.62       -3.82    2.0    259ms
  7   -7.926861680361       -8.01       -4.32    1.4    243ms
  8   -7.926861681744       -8.86       -4.83    1.8    253ms
  9   -7.926861681858       -9.94       -5.23    2.0    297ms
 10   -7.926861681871      -10.88       -5.90    1.8    250ms
 11   -7.926861681873      -11.83       -6.97    2.0    271ms
 12   -7.926861681873      -13.46       -7.65    3.5    319ms
 13   -7.926861681873      -14.75       -8.15    2.5    288ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921703754129                   -0.69    5.6         
  2   -7.926169760669       -2.35       -1.22    1.0    253ms
  3   -7.926839383035       -3.17       -2.37    1.8    294ms
  4   -7.926864903250       -4.59       -3.00    2.6    316ms
  5   -7.926865038912       -6.87       -3.31    2.0    319ms
  6   -7.926865077042       -7.42       -3.72    1.5    243ms
  7   -7.926865091787       -7.83       -4.40    1.4    242ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
